<a href="https://colab.research.google.com/github/gregrell/CAP6671/blob/main/CAP6671_CombinedArms_Greg_Rell_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This code example uses [PettingZoo](https://www.pettingzoo.ml/#) as a mutli-agent reinforcement learning platform for the [Combinded Arms](https://www.pettingzoo.ml/magent/combined_arms) environment.

This is an example of how to render PettingZoo's GUI in Colab and also serves as a template for testing separate Red and Blue models.

Author: Dr. S. Mondesire

Last Modified: 2022-04-03

# Import Libraries

In [1]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper
!apt update && apt install python-opengl ffmpeg
!pip install pygame
!pip install pettingzoo[magent]

!pip install stable_baselines3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

# Model Creation
This model randomly moves, attacks, or does nothing per timestep. Handles both ranged and melee agents.

In [ ]:
class MyModel:
  def predict(self, env, observation, agent):
    action = env.action_space(agent).sample()
    return action

# Test Model

In [ ]:
import gym
import gnwrapper
from pettingzoo.magent import combined_arms_v5
import pickle

from stable_baselines3 import PPO

MAX_TIMESTEPS = 500

# Initialize the Combined Arms environment
env = combined_arms_v5.parallel_env() # Parallel environemnt
env = gnwrapper.LoopAnimation(env) # Start Xvfb

# Instantiate models
redModel = MyModel()
blueModel = MyModel()

# Save (pickle) model
pickle.dump(redModel, open("redModel.pkl", 'wb') )

# Load pickled model
with open("redModel.pkl", 'rb') as pickle_model:
  redModel = pickle.load(pickle_model)

observations = env.reset()


model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)

for step in range(MAX_TIMESTEPS):
  actions = {}

  # For each agent, get the next action
  for agent in env.agents:
    model = None
    if "red" in agent:
      model = redModel
    elif "blue" in agent:
      model = blueModel
    
    actions.update({agent: model.predict(env, observations[agent], agent) })

  observations, rewards, dones, infos = env.step(actions)
  
  env.render() # Render the current frame

# Display the rendered movie
env.display()

# Final Stats
redMelee = 0
redRanged = 0
blueMelee = 0
blueRanged = 0
for agent in env.agents:
  if "redmelee" in agent:
    redMelee += 1
  elif "redranged" in agent:
    redRanged += 1
  elif "bluemele" in agent:
    blueMelee += 1
  elif "blueranged" in agent:
    blueRanged += 1

print("Red Total {}: (Melee {}, Ranged {})".format(redMelee+redRanged,redMelee, redRanged) )
print("Blue Total {}: (Melee {}, Ranged {})".format(blueMelee+blueRanged,blueMelee, blueRanged) )



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AttributeError: ignored